<a href="https://colab.research.google.com/github/Kaia-nyoung/2025-ML-class/blob/main/10%EC%A3%BC%EC%B0%A8/AE_WISDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
csv_path = '/content/drive/MyDrive/ML-MK/WISDM.csv'   # 예시: 구글코랩이면 '/content/WISDM.csv' 식으로

# WISDM 원본 포맷이 보통 header가 없으니까 우리가 이름을 붙여준다.
# (user, activity, timestamp, x, y, z) 구조를 많이 쓰므로 이렇게 가정
col_names = ['user', 'activity', 'timestamp', 'x', 'y', 'z']
df = pd.read_csv(csv_path, header=None, names=col_names)

In [3]:
# 3. 기본 전처리 -------------------------------------------------
# 숫자열로 변환 (문자 들어간 거 있으면 NaN으로)
for col in ['x', 'y', 'z']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# x,y,z 중 하나라도 NaN이면 제외
df = df.dropna(subset=['x', 'y', 'z']).reset_index(drop=True)

In [4]:
# 4. 라벨 만들기 -------------------------------------------------
# 오토인코더 예제들은 보통 "정상 0 / 이상 1" 구조를 쓴다.
# 여기서는 가장 많이 나온 activity를 "정상(0)"으로 보고 나머지를 전부 "이상(1)"으로 본다.
top_activity = df['activity'].value_counts().idxmax()
df['Class'] = (df['activity'] != top_activity).astype(int)

print("가장 많이 나온 활동을 정상(0)으로 사용합니다 ->", top_activity)
print(df['Class'].value_counts())

가장 많이 나온 활동을 정상(0)으로 사용합니다 -> Walking
Class
1    641800
0    406775
Name: count, dtype: int64


In [5]:
# 5. 특징 선택 + 스케일링 ----------------------------------------
features = ['x', 'y', 'z']      # 센서 3축만 사용
X = df[features].values
y = df['Class'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 오토인코더는 정상 데이터만으로 학습
X_train = X_scaled[y == 0]   # 정상(0)인 애들만
X_test = X_scaled            # 전체로 성능 확인
y_test = y                   # 실제 라벨

input_dim = X_train.shape[1]  # 보통 3

In [6]:
# 6. 오토인코더 모델 정의 ----------------------------------------
def build_autoencoder(input_dim):
    inp = layers.Input(shape=(input_dim,))
    # encoder
    x = layers.Dense(16, activation='relu')(inp)
    x = layers.Dense(8, activation='relu')(x)
    encoded = layers.Dense(4, activation='relu')(x)
    # decoder
    x = layers.Dense(8, activation='relu')(encoded)
    x = layers.Dense(16, activation='relu')(x)
    out = layers.Dense(input_dim, activation='linear')(x)

    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='mse')
    return model

autoencoder = build_autoencoder(input_dim)
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 471 (1.84 KB)

 Trainable params: 471 (1.84 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# 7. 학습 --------------------------------------------------------
history = autoencoder.fit(
    X_train, X_train,
    epochs=30,            # 필요하면 늘려도 됨
    batch_size=256,
    validation_split=0.1,
    verbose=1
)


Epoch 1/30
1431/1431 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.1796 - val_loss: 4.8977e-04
Epoch 2/30
1431/1431 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1.8385e-04 - val_loss: 2.5272e-05
Epoch 3/30
1431/1431 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 2.1773e-05 - val_loss: 5.0598e-06
Epoch 4/30
1431/1431 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 1.0013e-05 - val_loss: 3.0191e-05
Epoch 5/30
1431/1431 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 2.7418e-05 - val_loss: 8.1837e-07
Epoch 6/30
1431/1431 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 7.4022e-06 - val_loss: 6.5521e-06
Epoch 7/30
1431/1431 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 1.1129e-05 - val_loss: 2.8482e-06
Epoch 8/30
1431/1431 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 1.6685e-05 - val_loss: 5.0636e-07
Epoch 9/30
1431/1431 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.1333e-05 - val_loss: 2.4736e-07
Epoch 10/30
1431/1431 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 8.3352e-06 - val_loss: 1.0768e-05
Epoch 11/30
1431/1431 ━━━━━━━━━━━━━━━━━━━━ 

In [8]:
# 8. 정상 데이터의 오차 분포로 임계값 설정 ------------------------
# 정상으로 학습한 애들(X_train)에 대해 복원오차 계산
recon_train = autoencoder.predict(X_train)
mse_train = np.mean(np.power(X_train - recon_train, 2), axis=1)

# 예시: 상위 95% 지점을 threshold로 사용
threshold = np.percentile(mse_train, 95)
print("선택된 임계값(threshold):", threshold)

12712/12712 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step
선택된 임계값(threshold): 3.8162579559999e-08


In [9]:
# 9. 전체 데이터에 적용해서 이상탐지 ------------------------------
recon_test = autoencoder.predict(X_test)
mse_test = np.mean(np.power(X_test - recon_test, 2), axis=1)

# 임계값보다 크면 이상(1), 아니면 정상(0)
y_pred = (mse_test > threshold).astype(int)

32768/32768 ━━━━━━━━━━━━━━━━━━━━ 47s 1ms/step


In [10]:
# 10. 평가 -------------------------------------------------------
# 실제 라벨이 0,1 둘 다 있으면 평가를 출력
if len(np.unique(y_test)) > 1:
    print("\n=== Confusion Matrix ===")
    print(confusion_matrix(y_test, y_pred))
    print("\n=== Classification Report ===")
    print(classification_report(y_test, y_pred, digits=4))
else:
    print("\n(주의) 실제 라벨이 한 클래스만 있어서 평가표는 의미가 적습니다.")


=== Confusion Matrix ===
[[386434  20341]
 [557024  84776]]

=== Classification Report ===
              precision    recall  f1-score   support

           0     0.4096    0.9500    0.5724    406775
           1     0.8065    0.1321    0.2270    641800

    accuracy                         0.4494   1048575
   macro avg     0.6080    0.5410    0.3997   1048575
weighted avg     0.6525    0.4494    0.3610   1048575



In [11]:
error_df = pd.DataFrame({
    'reconstruction_error': mse_test,
    'true_class': y_test,
    'pred_class': y_pred,
    'activity': df['activity']
})

print("\n오류가 큰 상위 10개")
print(error_df.sort_values('reconstruction_error', ascending=False).head(10))


오류가 큰 상위 10개
        reconstruction_error  true_class  pred_class activity
604134              0.180336           1           1  Jogging
604572              0.177976           1           1  Jogging
209555              0.141113           1           1  Jogging
210839              0.140776           1           1  Jogging
213441              0.140114           1           1  Jogging
213450              0.140037           1           1  Jogging
163045              0.139933           1           1  Jogging
603678              0.139899           1           1  Jogging
208871              0.139897           1           1  Jogging
209865              0.139893           1           1  Jogging
